# Earth Engine Code for extraction of timeseries

In [ ]:

// Define the start and end dates for the time series period
var startDate = '2002-01-01';  // Start of the time series
var endDate = '2022-01-01';    // End of the time series



// Define the dataset
var datasetCollection = ee.ImageCollection('NASA/GRACE/MASS_GRIDS_V04/LAND')
  .filterDate(startDate, endDate); // Filter by date range
  
  
print(datasetCollection)

// Define the variable for equivalent water thickness (lwe_thickness)
var lwe_thickness = datasetCollection.select('lwe_thickness_csr');

print(lwe_thickness)

// Define a function to extract statistics for each image in the collection
var extractStats = function(image) {
  var stats = image.reduceRegion({
    reducer: ee.Reducer.mean(),
    geometry: table,
    scale: 25000,
    maxPixels: 1e8
  });
  
  // Create a feature with the statistics and the date
  var date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd');  // Format the date
  var feature = ee.Feature(null, stats.set('date', date));
  
  return feature;
};

// Map the extraction function over the image collection
var statsCollection = lwe_thickness.map(extractStats);

// Print the stats collection to see the result
print(statsCollection);




// Convert to a feature collection for easier handling
var statsList = statsCollection.map(function(feature) {
  return ee.Feature(null, feature.toDictionary());
});

// Create a time series chart for the variation of equivalent water thickness over time
var chart = ui.Chart.feature.byFeature(statsList, 'date', ['lwe_thickness_csr'])
  .setChartType('LineChart')
  .setOptions({
    title: 'Time Series of Equivalent Water Thickness',
    hAxis: {
      title: 'Date',
      format: 'YYYY-MM-dd'
    },
    vAxis: {
      title: 'Equivalent Liquid Water Thickness (m)',
      minValue: -1,
      maxValue: 1
    },
    lineWidth: 2,
    pointSize: 4
  });

// Print the chart
print(chart);
